## Colour and shape detection 
# Overview 
- Colour detection is done first 
- This is followed up with Shape detection

In [1]:
import cv2 
import numpy as np

In [2]:
# import required libraries
import numpy as np
import cv2
red = np.uint8([[[40, 120, 255]]])

# here insert the bgr values which you want to convert to hsv
bgr2 = np.uint8([[[40,120,255]]])
bgr = np.uint8([[[38,0,217]]])
hsv = cv2.cvtColor(bgr, cv2.COLOR_BGR2HSV)
hsv2 = cv2.cvtColor(bgr2, cv2.COLOR_BGR2HSV)
print("BGR Value:", bgr)
print("BGR Value:", bgr2)
print("HSV Value 1:", hsv)
print("HSV Value 2:", hsv2)

# compute the lower and upper limits
lowerLimit = bgr[0][0][0] - 10, 100, 100
upperLimit = bgr[0][0][0] + 10, 255, 255

# display the lower and upper limits
print("Lower Limit:",lowerLimit)
print("Upper Limit", upperLimit)

BGR Value: [[[ 38   0 217]]]
BGR Value: [[[ 40 120 255]]]
HSV Value 1: [[[175 255 217]]]
HSV Value 2: [[[ 11 215 255]]]
Lower Limit: (28, 100, 100)
Upper Limit (48, 255, 255)


In [3]:
class ColorDetector: 
    def __init__(self): 
        pass
    def detectColor(self, img): 
        hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        lower_red = np.array([0, 100, 170])
        upper_red = np.array([15, 255, 255])
        
        mask = cv2.inRange(hsv_img, lower_red, upper_red)
        result = cv2.bitwise_and(img, img, mask = mask)
        kernel = np.ones((10,10), np.uint8)
        erosion = cv2.erode(result, kernel, iterations = 1)
        dilation = cv2.dilate(erosion, kernel, iterations = 1)
        
        img_gray = cv2.cvtColor(dilation, cv2.COLOR_BGR2GRAY) #convert the image to grayscale, this is a requirement bfr thresholding
        _, thresh = cv2.threshold(img_gray, 70, 255, cv2.THRESH_BINARY)
        contours, hierarchy = cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
        
        img_contours = cv2.drawContours(img, contours, -1 , (0, 255, 0), 3) #green contours
        for c in contours: 
            rect = cv2.boundingRect(c)
            x, y, w, h = rect 
            cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 3)
        return img, contours
        

In [4]:
# img = cv2.imread('soccerballs.png') 
# s = ColorDetector()
# final_img, contours = s.detectColor(img)

# #classified_img = contour_loop(contours, final_img)
# cv2.imshow('Image',final_img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [5]:
class ShapeDetector: 
    def __init__(self):
        pass 
    def detect(self, c): 
        shape = "unidentified"
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.04 * peri, True)
        # if the shape is a triangle, it will have 3 vertices
        if len(approx) == 3:
            shape = "triangle"
        # if the shape has 4 vertices, it is either a square or
        # a rectangle
        elif len(approx) == 4:
            # compute the bounding box of the contour and use the
            # bounding box to compute the aspect ratio
            (x, y, w, h) = cv2.boundingRect(approx)
            ar = w / float(h)
            # a square will have an aspect ratio that is approximately
            # equal to one, otherwise, the shape is a rectangle
            shape = "square" if ar >= 0.95 and ar <= 1.05 else "rectangle"
        # if the shape is a pentagon, it will have 5 vertices
        elif len(approx) == 5:
            shape = "pentagon"
        # otherwise, we assume the shape is a circle
        else:
            shape = "circle"
        # return the name of the shape
        return shape

In [6]:
# Define a function to calculate the distance between two points
def calculate_distance(point1, point2):
    return ((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2) ** 0.5

def contour_loop(contour, image):
    sd = ShapeDetector()
    print(len(contour))
    Arr = []
    
    for c in contour:
        # These 3 lines find the (x,y) for the centre of an object
        M = cv2.moments(c)
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
        
        # Check if the new (cX, cY) is within a 10-pixel radius of any point in Arr
        is_duplicate = False
        for point in Arr:
            if calculate_distance((cX, cY), point) < 10:
                is_duplicate = True
                break

        if not is_duplicate:
            Arr.append((cX, cY))
            shape = sd.detect(c)
            print(shape)
            cv2.putText(image, shape, (cX, cY), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)

    return image

In [ ]:
img = cv2.imread('soccerballs.png') 
s = ColorDetector()
final_img, contours = s.detectColor(img)

classified_img = contour_loop(contours, final_img)
cv2.imshow('Image',final_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

4
circle
[(135, 200)]
